<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing an Application Built Using Groq</h1>

Groq api is used for faster inference and greater efficiency when calling hosted LLMs. Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your query engine and surfacing problematic `spans` of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Perform API requests using groq to answer questions
- Record trace data in [OpenInference tracing](https://github.com/Arize-ai/openinference) format using the global `arize_phoenix` handler
- Inspect the traces and spans of your application to identify sources of latency and cost

ℹ️ This notebook requires a Groq API key.

# 1. Install Dependencies and Setup Environment
Install Phoenix, Groq, and Arize-Otel.  Set API keys as environment variables.

In [ ]:
! pip install -q openinference-instrumentation-groq groq arize-phoenix

In [ ]:
import os
from getpass import getpass

if not (groq_api_key := os.getenv("GROQ_API_KEY")):
    groq_api_key = getpass("🔑 Enter your Groq API key: ")

os.environ["GROQ_API_KEY"] = groq_api_key

## 2. Launch Phoenix and Configure Auto-Instrumentation

Register local phoenix as the otel endpoint using `register`.  Launch phoenix locally and collect traces for each request in the session.

In [ ]:
import phoenix as px
from phoenix.otel import register

session = px.launch_app()
register()

With auto-instrumentation from openinference, preparing to capture requests to the Groq API using open-telemetry only takes a single line:

`GroqInstrumentor().instrument()`

In [ ]:
from openinference.instrumentation.groq import GroqInstrumentor

GroqInstrumentor().instrument()

## 3. Send Requests to LLM Hosted through Groq

The following code instantiates the Groq client and sends a request to __mixtral-8x7b-32768__ LLM hosted in Groq.  The openinference auto-instrumentor will capture the request and response and send them to the designated endpoint - in this case, the local phoenix service.

In [ ]:
from groq import Groq

# Groq client automatically picks up API key
client = Groq()

questions = [
    "Explain the importance of low latency LLMs",
    "What is Arize Phoenix and why should I use it?",
    "Is Groq less expensive than hosting on Amazon S3?",
]

# Requests sent to LLM through Groq client are traced to Phoenix
chat_completions = [
    client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
        model="mixtral-8x7b-32768",
    )
    for question in questions
]

for chat_completion in chat_completions:
    print("\n------\n" + chat_completion.choices[0].message.content)

# Launch the Phoenix UI if you haven't already

In [ ]:
print("Launch phoenix here if you haven't already: ", session.url)